# Test RedisAI with Pytorch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Create the network

Take the network from the pytorch MNIST examples 
(https://github.com/pytorch/examples/blob/master/mnist/main.py)

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

## Create the dataset and the data loader

In [4]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

data = datasets.MNIST('./data', train=True, download=False, transform=transform)

In [5]:
train_loader = torch.utils.data.DataLoader(data, batch_size=1)

In [6]:
it = iter(train_loader)

### Load a single tensor to forward

In [7]:
img, target = it.next()

## Create the network and do a forward and backward pass to get the gradients

In [8]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.01)
model.train()

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [12]:
# Zero the optimizer before the forward pass
optimizer.zero_grad()

# forward pass
out = model(img)

loss = F.nll_loss(out, target)
loss.backward()

In [13]:
c1_grad = model.conv1.weight.grad

In [18]:
c1_grad.numpy()

array([[[[ 3.33329570e-03, -9.70054604e-03, -1.58126224e-02],
         [-2.12285183e-02, -4.86872271e-02, -2.60166563e-02],
         [-2.90685631e-02, -3.35905068e-02, -1.84086785e-02]]],


       [[[-1.41874310e-02, -1.36032440e-02, -1.73040517e-02],
         [-1.08043617e-02,  3.17359716e-03,  2.31757062e-03],
         [ 1.72783658e-02,  2.18862034e-02,  1.95861794e-02]]],


       [[[-1.10776210e-02, -1.94403306e-02, -1.62097141e-02],
         [ 1.17441360e-02, -1.56387128e-03, -4.90442524e-03],
         [ 2.40342394e-02,  1.50080454e-02,  1.22816805e-02]]],


       [[[ 1.27604790e-02,  1.36497412e-02, -4.33015637e-04],
         [ 4.12298553e-02,  2.68132091e-02,  1.64469257e-02],
         [ 1.70850232e-02, -1.23672821e-02, -2.00330429e-02]]],


       [[[-2.23137178e-02, -1.98904872e-02, -3.89945060e-02],
         [-1.62129402e-02, -3.70368026e-02, -5.64495586e-02],
         [-2.06521731e-02, -2.43385006e-02, -5.11687323e-02]]],


       [[[-1.98822953e-02, -2.51425542e-02, -1.448

## Start with the REDIS AI part

In [15]:
import redisai as rai

In [16]:
con = rai.Client(host='192.168.99.102', port=6379)

In [19]:
con.tensorset('grad-conv1', c1_grad.numpy(), dtype='float32')

'OK'